In [6]:
import pandas as pd
import re
from typing import List, Tuple, Dict, Set
import logging
from collections import defaultdict
import random

from main import *
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

In [2]:
# input_path  = r"G:\PROJ\220927\戎圩填字表用_方音調搽字表_220928.xlsx"
# locale_name = "戎墟"
# SHEET_NAME  = "主表"
# CHAR_COL    = 'H'
# PRON_COL    = "TUV"
# PRON_ND_COL = "WXY"
# MEAN_COL    = 'Z'

In [37]:
input_path  = r"C:\Users\52643\Desktop\順德大良 211021.xlsx"
locale_name = "端州"
sheet_name  = "Sheet1"
char_col    = 'A'
pron_col    = "B"
pron_nd_col = ""
mean_col    = ''
ipa_col     = ""

In [8]:
char_col_index    = [ord(char_col) - 65]
pron_col_index    = [ord(i) - 65 for i in pron_col]
pron_nd_col_index = [ord(i) - 65 for i in pron_nd_col]
mean_col_index    = [ord(i) - 65 for i in mean_col]
ipa_col_index     = [ord(i) - 65 for i in ipa_col]
use_col_index: Dict[str, List[int]] = {
    "char":char_col_index, "mean":mean_col_index, "ipa":ipa_col_index, "pron":pron_col_index, "pron_nd":pron_nd_col_index}

In [38]:
locale_rule = [0, 1, locale_name]
get_ipa_for_locale = lambda pron: get_ipa(locale_rule, pron, locale_name)

data_file = pd.read_excel(input_path, sheet_name=sheet_name, keep_default_na=False, dtype=str)

In [39]:
entry_list, chara_index_dict = read_sheet(data_file, use_col_index)
for entry in entry_list: entry.rm_duplicate()
# entry_list, chara_index_dict = sim_2_trad(entry_list, chara_index_dict, False)
# for entry in entry_list: entry.toIpa(locale_rule, locale_name)

[2023-08-01 19:32:12,095] INFO: 總共 3093 行
[2023-08-01 19:32:12,353] INFO: 讀取 3003 行


2821

In [40]:
entry_list[chara_index_dict['石']].multiprons[0].prons[0]

'sek6'

In [41]:
sorted(entry_list[chara_index_dict['石']].multiprons, key=lambda x:x.prons[0])

In [42]:
print(entry_list[1010])
print(entry_list[1010].multiprons[0].split_prons[0])

椒 => ['ziu153']
['z', 'iu', '', '153']


In [13]:
csv_file_path = 'data/KwangUon.csv'
df = pd.read_csv(csv_file_path,
                 usecols=["最簡描述", "字頭覈校前", "字頭"],
                 keep_default_na=False)
df.head()

,最簡描述,字頭覈校前,字頭
0,端一東平,東,
1,端一東平,菄,
2,端一東平,鶇,
3,端一東平,䍶,
4,端一東平,𠍀,


In [14]:
所有母 = '幫滂並明端透定泥來知徹澄孃精清從心邪莊初崇生俟章昌常書船日見溪羣疑影曉匣云以'
所有等 = '一二三四'
所有韻 = '東冬鍾江支脂之微魚虞模齊祭泰佳皆夬灰咍廢眞臻文欣元魂痕寒刪山仙先蕭宵肴豪歌麻陽唐庚耕清青蒸登尤侯幽侵覃談鹽添咸銜嚴凡'
所有聲 = '平上去入'
韻順序表 = '東_冬鍾江支脂之微魚虞模齊祭泰佳皆夬灰咍廢眞臻文欣元魂痕寒刪山仙先蕭宵肴豪歌_麻_陽唐庚_耕清青蒸登尤侯幽侵覃談鹽添咸銜嚴凡'
脣音母 = set('幫滂並明')
開合中立韻 = set('東冬鍾江虞模尤幽')
重紐母 = set('幫滂並明見溪羣疑影曉匣')
v1重紐韻 = set('支脂祭眞仙宵侵鹽清')
呼到韻 = {
    '開': set('咍痕欣嚴之魚臻蕭宵肴豪侯侵覃談鹽添咸銜'),
    '合': set('灰魂文凡'),
}
等到韻 = {
    '一': set('冬模泰咍灰痕魂寒豪唐登侯覃談'),
    '二': set('江佳皆夬刪山肴耕咸銜'),
    '三': set('鍾支脂之微魚虞祭廢眞臻欣元文仙宵陽清蒸尤幽侵鹽嚴凡'),
    '四': set('齊先蕭青添'),
}
攝到韻 = {
    "通":set("東冬鍾"),
    "江":set("江"),
    "止":set("支脂之微"),
    "遇":set("魚虞模"),
    "蟹":set("齊佳皆灰咍祭泰夬廢"),
    "臻":set("眞諄臻文欣元魂痕"),
    "山":set("寒桓刪山先仙"),
    "效":set("蕭宵肴豪"),
    "果":set("歌戈"),
    "假":set("麻"),
    "宕":set("唐陽"),
    "梗":set("庚耕清青"),
    "曾":set("登蒸"),
    "流":set("侯尤幽"),
    "深":set("侵"),
    "咸":set("覃談鹽添咸銜嚴凡")}
組到聲 = {
    "幫":set("幫滂並明"),
    "端":set("端透定泥"),
    "知":set("知徹澄孃"),
    "來":set("來"),
    "精":set("精清從心邪"),
    "莊":set("莊初崇生俟"),
    "章":set("章昌常書船"),
    "日":set("日"),
    "見":set("見溪羣疑"),
    "影":set("影云曉匣"),
    "以":set("以")}
PATTERN_描述 = re.compile(f'([{所有母}])([開合])?([{所有等}])?([AB])?([{所有韻}])([{所有聲}])')
取得地位 = lambda 描述: PATTERN_描述.fullmatch(描述).groups()

In [15]:
韻到等 = defaultdict()
for 等, 韻s  in 等到韻.items():
    for 韻 in 韻s: 韻到等[韻] = 等
韻到呼 = defaultdict()
for 呼, 韻s  in 呼到韻.items():
    for 韻 in 韻s: 韻到呼[韻] = 呼
for 韻 in 開合中立韻:
    韻到呼[韻] = "中"
韻到攝 = defaultdict()
for 攝, 韻s  in 攝到韻.items():
    for 韻 in 韻s: 韻到攝[韻] = 攝
聲到組 = defaultdict()
for 組, 聲s  in 組到聲.items():
    for 聲 in 聲s: 聲到組[聲] = 組

In [16]:
class Loc():
    聲: str = ""
    組: str = ""
    呼: str = ""
    等: str = ""
    紐: str = ""
    攝: str = ""
    韻: str = ""
    調: str = ""
    def __init__(self, loc: Tuple):
        self.聲, self.呼, self.等, self.紐, self.韻, self.調 = [l if l!=None else "" for l in loc]
        self.攝 = 韻到攝[self.韻]
        self.組 = 聲到組[self.聲]
        if self.等=="": self.等 = 韻到等[self.韻]
        if self.呼=="" and self.韻 in 韻到呼: self.呼 = 韻到呼[self.韻]
    def __str__(self):
        # return "聲: %s, 呼: %s, 等: %s, 紐: %s, 韻: %s, 調: %s, 攝: %s" % (self.聲, self.呼, self.等, self.紐, self.韻, self.調, self.攝)
        return f"{self.組}組{self.聲}母{self.攝}攝{self.等}等{self.韻}{self.紐}韻{self.調}聲"
    def get(self, name: str) -> str:
        if name=='聲': return self.聲
        elif name=='呼': return self.呼
        elif name=='等': return self.等
        elif name=='紐': return self.紐
        elif name=='韻': return self.韻
        elif name=='調': return self.調
        elif name=='攝': return self.攝
        elif name=='組': return self.組
        else: raise Exception("不支援的地位名稱: " + name)

In [17]:
def get_jpp_final(jpp_splited: List[str], rewrite_final=True):
    nuclei, final = jpp_splited[1:3]
    f = final
    if rewrite_final:
        if f=="p": f="m"
        if f=="t": f="n"
        if f=="k": f="v"
    return nuclei + f

In [18]:
chara_to_locate: Dict[str, List[Loc]] = defaultdict(list)
for i in df.itertuples():
    if not i.最簡描述: continue
    chara_head = i.字頭 if i.字頭!="" else i.字頭覈校前
    chara_to_locate[chara_head].append(Loc(取得地位(i.最簡描述)))
print(chara_to_locate['重'][0])

知組澄母通攝三等鍾韻平聲


In [19]:
# for finals  : 紐<-呼<-韻<-攝<-聲<-調
# for initials: 

In [20]:
class PronNode():
    children: Dict[str, 'PronNode'] # name -> PronNode
    names   : str # names of the Loc # 本節點的地位
    type    : str # 'final' or 'initial' or what # 本節點的類型
    charas  : str # all charas that belongs to the Loc # 音韻地位下的轄字
    prons   : Dict[str, set] # pron of the charas # 時音-轄字
    def __init__(self, name: str, type: str) -> None:
        self.names    = name
        self.type     = type
        self.charas   = ""
        self.children = defaultdict(PronNode)
        self.prons    = defaultdict(set)
    def __repr__(self) -> str:
        return self.repr();
    def repr(self, show_charas=False):
        this_layer = self.names + "[" + self.type +"]: " + (self.charas if show_charas else "")
        if len(self.children)>0: this_layer += " { " + "".join([v.names for k, v in self.children.items()]) + " } "
        if len(self.prons)>0: this_layer += ("\n=時音: " if show_charas else "") + ", ".join(["%s: %s" % (k,"".join(v)) for k, v in self.prons.items()])
        return this_layer
    def __str__(self) -> str:
        return self.str()
    def str(self, indent_mark:str=" ", layer:int=0, show_charas=True) -> str:
        this_layer = (indent_mark*layer) + "-" + self.names + "[" + self.type +"]: " + (self.charas if show_charas else "")
        if len(self.prons)>0: this_layer += (("\n"+(indent_mark*(layer+1))+"=時音: ") if show_charas else "") + ", ".join(["%s: %s" % (k,"".join(v)) for k, v in self.prons.items()])
        if len(self.children) == 0: return this_layer
        next_layer = "\n".join(child.str(indent_mark, layer+1, show_charas) for child in self.children.values())
        return this_layer + "\n" + next_layer
    def fold(self, thre=0.8, fold_empty=False) -> None:
        id = random.randint(0, 1000000)
        print("FOLDING %07d: %s" % (id, self.__repr__()))
        if len(self.children) == 0: 
            pass # this is a leaf node
        elif len(self.children) == 1: # this node has only one child
            child_0 = self.children[list(self.children)[0]]
            child_0.fold(thre=thre, fold_empty=fold_empty)
            # if child_0.isleaf():
            self.charas = child_0.charas
            self.prons  = child_0.prons
            self.children = child_0.children
        elif len(self.children)>1:
            prons:Dict[str, set] = defaultdict(set)
            prons_chs = ""
            charas = ""
            need_fold = []
            is_empty = []
            for child in self.children.values():
                if not child.isleaf(): child.fold(thre=thre, fold_empty=fold_empty)
                if child.isleaf():
                    for pron, pron_chs in child.prons.items():
                        prons[pron] = prons[pron].union(pron_chs)
                        prons_chs += "".join(pron_chs)
                        charas += child.charas
                    need_fold.append(True)
                    is_empty.append(len(child.prons)==0)
                else:
                    need_fold.append(False)
                    is_empty.append(False)
                print("children: ", child.__repr__(), ", need_fold: ", need_fold[-1])
            print(f"need_fold: {need_fold}, is_empty: {is_empty}")
            if all(is_empty):
                self.charas = charas
                self.children = defaultdict(PronNode)
                self.prons = prons
                print("FOLD because of empty node")
            else:
                prons_percent = []
                for pron, pron_chs in prons.items():
                    prons_percent.append((len(pron_chs)+3)/(len(prons_chs)+3))
                prons_percent_above_thres = list(filter(lambda x:x>=thre, prons_percent))
                
                print(f"prons_chs: {prons_chs}, prons_percent: {prons_percent}, above_count: {len(prons_percent_above_thres)}")
                
                if (all(need_fold) and len(prons_percent_above_thres)>=1): # almost all children have the same pron
                    self.charas = charas
                    self.children = defaultdict(PronNode)
                    self.prons = prons
                    print("FOLD: " + ", ".join(prons.keys()))
        print("---- %07d" % id)
    def get(self, loc: Loc) -> 'PronNode':
        if len(self.children) == 0: return self
        for name, node in self.children.items():
            type_ = node.type
            if loc.get(type_) == name:
                return node.get(loc)
    def insert(self, pron: str, chara: str) -> None:
        # if pron not in self.prons:
        #     self.prons[pron] = set()
        self.prons[pron].add(chara)
    def build_node(self, chara: str, loc: Loc, order: List[str]):
        names = loc.get(order[0])
        if len(order) == 1:
            if names not in self.children:
                self.children[names] = PronNode(names, order[0])
                self.children[names].charas = chara
            else:
                self.children[names].charas += chara
        else:
            if names not in self.children:
                self.children[names] = PronNode(names, order[0])
            self.children[names].build_node(chara, loc, order[1:])
    def copy(self) -> 'PronNode':
        node = PronNode(self.names, self.type)
        node.charas = self.charas
        node.prons = self.prons.copy()
        for child in self.children.values():
            node.children[child.names] = child.copy()
        return node
    def isleaf(self) -> bool:
        return len(self.children) == 0
    def delete_empty(self) -> None:
        if self.isleaf(): return
        children = self.children.copy()
        for child in self.children:
            if children[child].isleaf() and len(children[child].prons)==0:
                del children[child]
            else:
                children[child].delete_empty()
                if children[child].isleaf() and len(children[child].prons)==0:
                    del children[child]
        self.children = children
        
    def get_prons(self) -> Dict[str, set]:
        if self.isleaf(): return self.prons
        prons = defaultdict(set)
        for child in self.children.values():
            for pron, pron_chs in child.get_prons().items():
                prons[pron] = prons[pron].union(pron_chs)
        return prons
    
    def __getattr__(self, __name: str) -> 'PronNode':
        return self.children[__name]

In [43]:
data_file = pd.read_excel(input_path, sheet_name=sheet_name, keep_default_na=False, dtype=str)
entry_list, chara_index_dict = read_sheet(data_file, use_col_index)
for entry in entry_list: entry.rm_duplicate()

[2023-08-01 19:32:29,528] INFO: 總共 3093 行
[2023-08-01 19:32:29,858] INFO: 讀取 3003 行


'i'

In [44]:
tree_final = PronNode("", "根")
for chara, locs in chara_to_locate.items():
    if len(locs)!=1: continue
    tree_final.build_node(chara, locs[0], ["聲","攝","等","韻","呼"]) #  # ["攝","韻","呼"]
for entry in entry_list:
    if len(entry.multiprons)==1 and len(entry.multiprons[0].prons)==1:
        chara = entry.chara
        pron  = entry.multiprons[0].split_prons[0]
        locs  = chara_to_locate[entry.chara]
        if len(locs) == 1: 
            tree_final.get(locs[0]).insert(pron[0], chara)
tree_final.delete_empty()

In [45]:
print(tree_final.str())

-[根]: 
 -端[聲]: 
  -通[攝]: 
   -一[等]: 
    -東[韻]: 
     -中[呼]: 東菄鶇𠍀倲𩜍𢘐鯟𢔅崠埬𧓕䰤董箽𢤦䵔𧄓棟㼯𩭩𧞐剢
      =時音: d: 董東棟
    -冬[韻]: 
     -中[呼]: 冬𠔙苳笗𩂓𧲴篤督𧝴𧞶錖𤬂𥓍
      =時音: d: 督冬篤
  -遇[攝]: 
   -一[等]: 
    -模[韻]: 
     -中[呼]: 都𥳉𧷿醏䩲覩睹暏賭帾妒妬秺蠹螙殬
      =時音: d: 賭妒都
  -蟹[攝]: 
   -四[等]: 
    -齊[韻]: 
     -開[呼]: 低袛磾鞮羝䧑奃鍉䐎𥿄㓳邸底䏄抵牴觝㪆軧帝諦嚏蔕螮𦨢骶僀䟡𤬵偙䩚渧
      =時音: d: 底抵低帝
   -一[等]: 
    -灰[韻]: 
     -合[呼]: 磓塠䭔堆鴭𢈹搥䜃嵟𩈜𠂤𡏩𦞙䇏𡑈𩬳對𡭊碓𣝉𠏮
      =時音: d: 堆碓對
    -咍[韻]: 
     -開[呼]: 懛戴
      =時音: d: 戴
    -泰[韻]: 
     -開[呼]: 帶𢄔艜㯂
      =時音: d: 帶
  -臻[攝]: 
   -一[等]: 
    -魂[韻]: 
     -合[呼]: 驐墩𤭞頓扽𩢎𩨳
      =時音: d: 墩頓扽
  -山[攝]: 
   -一[等]: 
    -寒[韻]: 
     -開[呼]: 襌鄲丹殫簞匰㠆䐷亶𤺺担旦鴠㡺𢛁妲炟䵣𦬹
      =時音: d: 丹旦
     -合[呼]: 端𧤗耑鍴𥵣𦾸短𢭃𢷖鍛腶碫瑖裰
      =時音: d: 鍛端短
   -四[等]: 
    -先[韻]: 
     -開[呼]: 㒹齻𩥄瘨癲巔𠑘傎蹎厧典蕇𥮏𠿍𥉺𨴗𡇓
      =時音: d: 典
  -效[攝]: 
   -四[等]: 
    -蕭[韻]: 
     -開[呼]: 貂𠚥刁琱凋鯛䂏雕鵰蛁彫𧘨𦶌𦨣㚋奝瞗鳥𢁕釕䄪𠄏伄瘹釣窵
      =時音: d: 釣刁貂雕, n: 鳥
   -一[等]: 
    -豪[韻]: 
     -開[呼]: 刀魛忉舠𩕯擣㨶島禂䮻壔到𤓾
      =時音: d: 到刀
  -果[攝]: 
   -一[等]: 
    -歌[韻]: 
     -開[呼]: 多𦰿𦷛嚲䯬䫂
      =時音: d: 多
   

In [37]:
tree_final.fold(thre=0.95, fold_empty=True)

FOLDING 0511864: [根]:  { 通江止遇蟹臻山效果假宕梗曾流深咸 } 
FOLDING 0765314: 通[攝]:  { 東冬鍾 } 
FOLDING 0839816: 東[韻]:  { 中 } 
FOLDING 0094783: 中[呼]:  { 一三 } 
FOLDING 0477588: 一[等]:  { 端定溪見明來匣從影清透精並曉心幫泥滂 } 
FOLDING 0530385: 端[聲]:  { 平上去 } 
children:  平[調]: uv: 鯟東菄倲鶇崠埬 , need_fold:  True
children:  上[調]: uv: 箽 , need_fold:  True
children:  去[調]: uv: 棟 , need_fold:  True
need_fold: [True, True, True], is_empty: [False, False, False]
prons_chs: 鯟東菄倲鶇崠埬箽棟, prons_percent: [1.0], above_count: 1
FOLD: uv
---- 0530385
children:  端[聲]: uv: 棟埬鯟東菄倲鶇崠箽 , need_fold:  True
FOLDING 0786196: 定[聲]:  { 平上去入 } 
children:  平[調]: uv: 犝同哃瞳銅仝 , need_fold:  True
children:  上[調]: uv: 動姛 , need_fold:  True
children:  去[調]: uv: 胴駧慟迵 , need_fold:  True
children:  入[調]: uv: 櫝殰瓄獨嬻豄讟黷犢贕髑牘皾韇匵 , need_fold:  True
need_fold: [True, True, True, True], is_empty: [False, False, False, False]
prons_chs: 犝同哃瞳銅仝動姛胴駧慟迵櫝殰瓄獨嬻豄讟黷犢贕髑牘皾韇匵, prons_percent: [1.0], above_count: 1
FOLD: uv
---- 0786196
children:  定[聲]: uv: 瓄嬻哃姛迵瞳讟黷銅動贕仝皾韇駧匵櫝殰犝獨胴同豄慟犢髑牘 , n

In [30]:
print(tree_final.str(show_charas=False))

-[根]: 
 -端[聲]: 
  -通[攝]: 
   -一[等]: 
    -東[韻]: 
     -中[呼]: ung: 董東棟
    -冬[韻]: 
     -中[呼]: ung: 冬, uv: 督篤
  -遇[攝]: 
   -一[等]: 
    -模[韻]: 
     -中[呼]: ou: 賭妬都妒睹
  -蟹[攝]: 
   -四[等]: 
    -齊[韻]: 
     -開[呼]: ai: 低牴抵帝底
   -一[等]: 
    -灰[韻]: 
     -合[呼]: ui: 堆碓對
    -咍[韻]: 
     -開[呼]: oi: 戴
    -泰[韻]: 
     -開[呼]: oi: 帶
  -臻[攝]: 
   -一[等]: 
    -魂[韻]: 
     -合[呼]: an: 頓扽
  -山[攝]: 
   -一[等]: 
    -寒[韻]: 
     -開[呼]: on: 丹妲旦鄲
     -合[呼]: yun: 鍛端短
   -四[等]: 
    -先[韻]: 
     -開[呼]: in: 癲, im: 典
  -效[攝]: 
   -四[等]: 
    -蕭[韻]: 
     -開[呼]: iu: 彫凋釣鳥刁貂雕
   -一[等]: 
    -豪[韻]: 
     -開[呼]: ou: 忉島到刀
  -果[攝]: 
   -一[等]: 
    -歌[韻]: 
     -開[呼]: u: 多
     -合[呼]: eo: 朵
  -宕[攝]: 
   -一[等]: 
    -唐[韻]: 
     -開[呼]: ong: 擋檔襠黨璫
  -梗[攝]: 
   -四[等]: 
    -青[韻]: 
     -開[呼]: eng: 頂叮酊鼎仃, ev: 的滴鏑
  -曾[攝]: 
   -一[等]: 
    -登[韻]: 
     -開[呼]: ang: 燈凳登, av: 德得
  -流[攝]: 
   -一[等]: 
    -侯[韻]: 
     -開[呼]: aau: 鬥陡篼抖蚪兜斗
  -咸[攝]: 
   -一[等]: 
    -覃[韻]: 
     -開[呼]: om: 答
    -談[韻]: 
     -開[呼]: om: 膽
   -四[等]: 
 

In [445]:
tree_ZEON = tree_final.children['臻'].children['魂'].copy()
# tree_ZEON = tree_final.children['臻'].copy()

In [453]:
tree_ZEON.fold(thre=0.7)

FOLDING 0529813: 魂[韻]:  { 匣見影明心精從端定清並幫溪曉泥 } 
children:  匣[聲]: an: 混餛棍 , need_fold:  True
children:  見[聲]: an: 骨昆 , need_fold:  True
children:  影[聲]: an: 穩溫 , need_fold:  True
children:  明[聲]: un: 悶沒門 , need_fold:  True
children:  心[聲]: yun: 孫遜損 , need_fold:  True
children:  精[聲]: yun: 尊 , need_fold:  True
children:  從[聲]: yun: 存蹲 , need_fold:  True
children:  端[聲]: an: 墩頓 , need_fold:  True
FOLDING 0158522: 定[聲]:  { 平上去 } 
children:  平[調]: yun: 豚臀, an: 飩 , need_fold:  True
children:  上[調]: yun: 囤 , need_fold:  True
children:  去[調]: an: 鈍 , need_fold:  True
need_fold: [True, True, True], is_empty: [False, False, False]
prons_chs: 豚臀飩囤鈍, prons_percent: [0.75, 0.625], above_count: 1
FOLD: yun, an
---- 0158522
children:  定[聲]: yun: 囤豚臀, an: 飩鈍 , need_fold:  True
children:  清[聲]: yun: 寸村忖, an: 猝 , need_fold:  True
children:  並[聲]: un: 勃脖餑𡋯盆 , need_fold:  True
children:  幫[聲]: un: 本 , need_fold:  True
children:  溪[聲]: an: 困坤窟 , need_fold:  True
children:  曉[聲]: an: 婚昏忽 , need_fold:  True
chi

In [448]:
tree_tmp = tree_ZEON.children['定']#.children['平']

In [489]:
set().union(*tree_tmp.prons.values()).__len__()

5

In [488]:
print(*tree_tmp.prons)

yun an


In [470]:
for loc, node in tree_ZEON.children.items():
    print(node.str(show_charas=False))
    print(list(node.prons.values()))

-匣[聲]: an: 混餛棍
[{'混', '餛', '棍'}]
-見[聲]: an: 骨昆
[{'骨', '昆'}]
-影[聲]: an: 穩溫
[{'穩', '溫'}]
-明[聲]: un: 悶沒門
[{'悶', '沒', '門'}]
-心[聲]: yun: 孫遜損
[{'孫', '遜', '損'}]
-精[聲]: yun: 尊
[{'尊'}]
-從[聲]: yun: 存蹲
[{'存', '蹲'}]
-端[聲]: an: 墩頓
[{'墩', '頓'}]
-定[聲]: yun: 囤豚臀, an: 飩鈍
[{'囤', '豚', '臀'}, {'飩', '鈍'}]
-清[聲]: yun: 寸村忖, an: 猝
[{'寸', '村', '忖'}, {'猝'}]
-並[聲]: un: 勃脖餑𡋯盆
[{'勃', '脖', '餑', '𡋯', '盆'}]
-幫[聲]: un: 本
[{'本'}]
-溪[聲]: an: 困坤窟
[{'困', '坤', '窟'}]
-曉[聲]: an: 婚昏忽
[{'婚', '昏', '忽'}]
-泥[聲]: yun: 嫩
[{'嫩'}]


In [408]:
tree_tmp.prons

defaultdict(set, {'yun': {'臀', '豚'}, 'an': {'飩'}})

In [ ]:
# -平[調]: yun: 豚臀, an: 飩
# -上[調]: yun: 囤
# -去[調]: an: 鈍



In [ ]:
to_be_merged: List[List[str]] = []

In [383]:
[len(node.prons) for node in tree_ZEON.children.values()]

[1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1]